In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib as plt

from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [53]:
titanic = pd.read_csv("titanic.train.csv")
titanic.describe()

print titanic.describe()
titanic = titanic.drop(['Ticket','Cabin','Name','PassengerId'], axis=1)
# Remove NaN values
titanic = titanic.dropna()

print titanic.describe()


       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
         Survived      Pclass         Age       SibSp       Parch        Fare
cou

In [54]:
for col in titanic.select_dtypes(exclude=["number"]).columns:
    print("Converting column "+col+"...")
    titanic[col] = titanic[col].astype('category')    
    print(titanic[col].cat.categories)
    titanic[col] = titanic[col].cat.codes   


Converting column Sex...
Index([u'female', u'male'], dtype='object')
Converting column Embarked...
Index([u'C', u'Q', u'S'], dtype='object')


In [55]:
X = titanic.copy()
y = titanic["Survived"]
X=X.drop("Survived", axis=1)
print X.shape
print X.describe()
svm_clf = Pipeline((
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ))

#svm_clf.fit(X_scaled, y)
svm_clf.fit(X, y)

(712, 7)
           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  712.000000  712.000000  712.000000  712.000000  712.000000  712.000000   
mean     2.240169    0.636236   29.642093    0.514045    0.432584   34.567251   
std      0.836854    0.481420   14.492933    0.930692    0.854181   52.938648   
min      1.000000    0.000000    0.420000    0.000000    0.000000    0.000000   
25%      1.000000    0.000000   20.000000    0.000000    0.000000    8.050000   
50%      2.000000    1.000000   28.000000    0.000000    0.000000   15.645850   
75%      3.000000    1.000000   38.000000    1.000000    1.000000   33.000000   
max      3.000000    1.000000   80.000000    5.000000    6.000000  512.329200   

         Embarked  
count  712.000000  
mean     1.595506  
std      0.779038  
min      0.000000  
25%      2.000000  
50%      2.000000  
75%      2.000000  
max      2.000000  


Pipeline(steps=(('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))))

In [74]:
test = pd.read_csv("titanic.test.csv")
test_x = test.drop(['Ticket','Cabin','Name','PassengerId'], axis=1)

print test_x.describe()
for col in test_x.select_dtypes(exclude=["number"]).columns:
    print("Converting column "+col+"...")
    test_x[col] = test_x[col].astype('category')    
    print(test_x[col].cat.categories)
    test_x[col] = test_x[col].cat.codes   

test_x.Age.fillna(value=test_x.Age.mean(), inplace=True)
test_x.Fare.fillna(value=test_x.Fare.mean(), inplace=True)
test_x.Embarked.fillna(value=(test_x.Embarked.value_counts().idxmax()), inplace=True)

print test_x.describe()
print test_x.shape

y_pred = svm_clf.predict(test_x)
print y_pred.shape
y_out = test[['PassengerId']].copy()
y_out["Survived"] = pd.DataFrame(y_pred,columns=["Survived"])
print y_out.describe()
print y_out[:10]
y_out.to_csv("titanic.pred.csv",index=False)


           Pclass         Age       SibSp       Parch        Fare
count  418.000000  332.000000  418.000000  418.000000  417.000000
mean     2.265550   30.272590    0.447368    0.392344   35.627188
std      0.841838   14.181209    0.896760    0.981429   55.907576
min      1.000000    0.170000    0.000000    0.000000    0.000000
25%      1.000000   21.000000    0.000000    0.000000    7.895800
50%      3.000000   27.000000    0.000000    0.000000   14.454200
75%      3.000000   39.000000    1.000000    0.000000   31.500000
max      3.000000   76.000000    8.000000    9.000000  512.329200
Converting column Sex...
Index([u'female', u'male'], dtype='object')
Converting column Embarked...
Index([u'C', u'Q', u'S'], dtype='object')
           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     2.265550    0.636364   30.272590    0.447368    0.392344   35.627188   
std      0.841838    0